# AI Agents Crash Course - Part 1 Implementation

This notebook demonstrates the key concepts from the Daily Dose of Data Science AI Agents article by Avi Chawla and Akshay Pachaar.

## What you'll learn:
1. **Single Agent Implementation** - Basic agent creation
2. **Multi-Agent Systems** - Collaborative workflows
3. **Tool Integration** - External capabilities
4. **Building Blocks** - The 6 fundamental components
5. **YAML Configuration** - Production-ready setup

## Prerequisites:
- Install required packages
- Set up API keys
- Choose your LLM provider

## 📦 Installation and Setup

First, let's install the required packages and set up our environment.

In [1]:
# Install required packages (run this cell first)
#!pip install crewai crewai-tools python-dotenv pyyaml IPython

In [ ]:
import os
import yaml
from dotenv import load_dotenv
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')

# CrewAI imports
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, FileReadTool

print("✅ All packages imported successfully!")

## 🔑 Environment Configuration

Set up your environment variables. You have two options:

### Option 1: Create a `.env` file with:
```
SERPER_API_KEY="your-serper-api-key"
AZURE_OPENAI_API_KEY="your-azure-openai-key"
AZURE_OPENAI_ENDPOINT="your-azure-openai-endpoint"
AZURE_OPENAI_API_VERSION="your-azure-openai-api-version"
AZURE_OPENAI_MODEL_NAME="your-azure-openai-model-name"
```

### Option 2: Set them directly in this notebook (not recommended for production):

In [ ]:
import os

# Load environment variables
load_dotenv()

# Option 2: Uncomment and set your API keys directly (not recommended for production)
# os.environ['OPENAI_API_KEY'] = 'your-openai-api-key-here'
# os.environ['SERPER_API_KEY'] = 'your-serper-api-key-here'

# Configure LLM - Choose one of the options below:

# Option A: Local Ollama (as mentioned in the article)
#llm = LLM(
#    model="ollama/llama3.2:1b",
#    base_url="http://localhost:11434"
#)

# Option B: OpenAI GPT-4 (uncomment to use)
# llm = LLM(model="gpt-4")

# Option C: OpenAI GPT-3.5-turbo (cheaper alternative)
# llm = LLM(model="gpt-3.5-turbo")

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## 🤖 Example 1: Single Agent - Technical Writer

Let's start with the simplest example: a single agent that acts as a technical writer.

### Key Concepts Demonstrated:
- **Role-playing**: Clear identity and expertise
- **Goal setting**: Specific objectives
- **Backstory**: Context and personality

In [ ]:
def create_technical_writer():
    """Create a senior technical writer agent"""
    return Agent(
        role="Senior Technical Writer",
        goal="Craft clear, engaging, and well-structured technical content based on research findings",
        backstory="""You are an experienced technical writer with expertise in simplifying complex 
                    concepts, structuring content for readability, and ensuring accuracy in documentation.
                    You have worked with leading tech companies and have a knack for making technical
                    topics accessible to diverse audiences.""",
        llm=llm,
        verbose=True
    )

# Create the agent
technical_writer = create_technical_writer()

# Create a writing task
writing_task = Task(
    description="Write a well-structured, engaging, and technically accurate article on {topic}.",
    agent=technical_writer,
    expected_output="A polished, detailed, and easy-to-read article on the given topic."
)

# Create and run the crew
single_agent_crew = Crew(
    agents=[technical_writer],
    tasks=[writing_task],
    verbose=True
)

print("✅ Single agent setup complete!")

In [ ]:
# Run the single agent example
print("🚀 Running Single Agent Example...")
single_agent_result = single_agent_crew.kickoff(inputs={"topic": "Introduction to AI Agents"})

# Display the result
display(Markdown("## Single Agent Result:"))
display(Markdown(str(single_agent_result)))

## 🔍 Example 2: Multi-Agent Research System

Now let's create a more complex system with multiple specialized agents working together.

### Agents in this system:
1. **Research Agent** - Finds information using web search
2. **Summarization Agent** - Condenses findings
3. **Fact-Checking Agent** - Verifies accuracy

### Key Concepts:
- **Specialization**: Each agent has a focused role
- **Tool Integration**: Web search capabilities
- **Sequential Workflow**: Tasks depend on each other

In [ ]:
# Initialize tools (only if you have SERPER_API_KEY)
try:
    serper_dev_tool = SerperDevTool()
    tools_available = True
    print("✅ SerperDev tool initialized successfully!")
except Exception as e:
    print(f"⚠️ SerperDev tool not available: {e}")
    print("💡 You can still run other examples without web search")
    tools_available = False

In [ ]:
def create_research_agents():
    """Create the research team agents"""
    
    # Agent 1: Research Agent
    research_agent = Agent(
        role="Internet Researcher",
        goal="Find the most relevant and recent information about a given topic.",
        backstory="""You are a skilled researcher, adept at navigating the internet 
                    and gathering high-quality, reliable information from online sources.
                    You excel at identifying credible sources and extracting key insights.""",
        tools=[serper_dev_tool] if tools_available else [],
        verbose=True,
        llm=llm
    )
    
    # Agent 2: Summarization Agent
    summarizer_agent = Agent(
        role="Content Summarizer",
        goal="Condense the key insights from research into a short and informative summary.",
        backstory="""You are an expert in distilling complex information into concise, 
                    easy-to-read summaries. You have a talent for identifying the most
                    important points and presenting them clearly.""",
        verbose=True,
        llm=llm
    )
    
    # Agent 3: Fact-Checking Agent
    fact_checker_agent = Agent(
        role="Fact-Checking Specialist",
        goal="Verify the accuracy of information and remove any misleading or false claims.",
        backstory="""You are an investigative journalist with a knack for validating facts, 
                    ensuring that only accurate information is published. You cross-reference
                    multiple sources and have a keen eye for misinformation.""",
        tools=[serper_dev_tool] if tools_available else [],
        verbose=True,
        llm=llm
    )
    
    return research_agent, summarizer_agent, fact_checker_agent

# Create the research team
if tools_available:
    research_agent, summarizer_agent, fact_checker_agent = create_research_agents()
    print("✅ Research team created successfully!")
else:
    print("⚠️ Skipping research team creation (no web search tools available)")

In [ ]:
# Run the multi-agent research system
if tools_available:
    print("🚀 Running Multi-Agent Research System...")
    
    # Create tasks
    research_task = Task(
        description="""Use available tools to search for the most relevant and recent data about {topic}.
                      Extract the key insights from multiple sources. Focus on recent developments,
                      trends, and authoritative information.""",
        agent=research_agent,
        tools=[serper_dev_tool],
        expected_output="A detailed research report with key insights and source references."
    )
    
    summarization_task = Task(
        description="""Summarize the research report into a concise and informative paragraph. 
                      Ensure clarity, coherence, and completeness. Highlight the most important
                      findings and trends.""",
        agent=summarizer_agent,
        expected_output="A well-structured summary with the most important insights."
    )
    
    fact_checking_task = Task(
        description="""Cross-check the summarized information for accuracy and remove any misleading claims.
                      Verify key facts using available tools. Ensure all information is credible
                      and up-to-date.""",
        agent=fact_checker_agent,
        tools=[serper_dev_tool],
        expected_output="A fact-checked and verified research summary."
    )
    
    # Create research crew with sequential process
    research_crew = Crew(
        agents=[research_agent, summarizer_agent, fact_checker_agent],
        tasks=[research_task, summarization_task, fact_checking_task],
        process=Process.sequential,
        verbose=True
    )
    
    # Execute the research workflow
    research_result = research_crew.kickoff(inputs={"topic": "Latest developments in AI agents 2025"})
    
    # Display results
    display(Markdown("## Multi-Agent Research Result:"))
    display(Markdown(str(research_result)))
else:
    print("⚠️ Skipping multi-agent research (requires SERPER_API_KEY)")
    research_result = "Multi-agent research skipped due to missing API key"

## 📁 Example 3: File Processing Agent

This example demonstrates how agents can work with files and documents.

### Key Concepts:
- **Tool Integration**: File reading capabilities
- **Document Processing**: Automated summarization
- **File I/O**: Reading and processing content

In [ ]:
def create_file_processing_example():
    """Create a file processing demonstration"""
    
    # Create a sample markdown file
    sample_content = """# AI Agents in Modern Applications

## Introduction
AI agents are autonomous systems that can perceive their environment, reason about what they perceive, and take actions to achieve specific goals.

## Key Capabilities
- **Autonomous Decision Making**: Agents can make decisions without human intervention
- **Tool Integration**: They can use external tools and APIs
- **Multi-Agent Collaboration**: Multiple agents can work together
- **Adaptive Learning**: Agents can improve their performance over time

## Applications
1. Customer service automation
2. Content creation and curation  
3. Research and data analysis
4. Process automation
5. Personal assistants

## Challenges
- Ensuring reliability and accuracy
- Managing computational costs
- Ethical considerations
- Integration complexity

## Conclusion
AI agents represent a significant advancement in automation technology, offering new possibilities for intelligent task automation across various domains.
"""
    
    # Write to file
    with open("ai_agents_sample.md", "w") as f:
        f.write(sample_content)
    
    print("✅ Sample file created: ai_agents_sample.md")
    return "ai_agents_sample.md"

# Create sample file
sample_file = create_file_processing_example()

In [ ]:
# Create file processing agent
file_read_tool = FileReadTool()

summarizer_agent = Agent(
    role="Senior Document Summarizer",
    goal="Extract and summarize key insights from provided files in a concise format.",
    backstory="""You are an expert in document analysis, skilled at extracting 
                key details, summarizing content, and identifying critical insights 
                from structured and unstructured text. You excel at creating
                executive summaries and highlighting the most important information.""",
    tools=[file_read_tool],
    verbose=True,
    llm=llm
)

# Create summarization task
file_summarizer_task = Task(
    description="""Use the FileReadTool to read the contents of {file_path}
                  and provide a comprehensive summary. Include:
                  1. Main topic and purpose
                  2. Key points and insights
                  3. Important conclusions
                  Ensure the summary captures the essence of the document.""",
    agent=summarizer_agent,
    tools=[file_read_tool],
    expected_output="A comprehensive summary highlighting the document's key insights and main points."
)

# Create and run crew
file_processing_crew = Crew(
    agents=[summarizer_agent],
    tasks=[file_summarizer_task],
    verbose=True
)

print("✅ File processing agent ready!")

In [ ]:
# Run file processing example
print("🚀 Running File Processing Example...")

file_result = file_processing_crew.kickoff(inputs={"file_path": sample_file})

# Display results
display(Markdown("## File Processing Result:"))
display(Markdown(str(file_result)))

# Clean up
if os.path.exists(sample_file):
    os.remove(sample_file)
    print(f"🧹 Cleaned up: {sample_file}")

## 📊 Results Summary

Congratulations! You've completed the AI Agents Crash Course Part 1 implementation.

In [ ]:
display(Markdown("""
# 🎯 AI Agents Implementation Summary

## Examples Completed:

### 1. ✅ Single Agent Technical Writer
- **Purpose**: Basic agent creation and task execution
- **Key Learning**: Simple role-based agents can produce quality content
- **Building Blocks Used**: Role-playing, Focus

### 2. {} Multi-Agent Research System  
- **Purpose**: Collaborative workflow with specialized agents
- **Key Learning**: Specialized agents work better than generalists
- **Building Blocks Used**: Role-playing, Focus, Tools, Cooperation

### 3. ✅ File Processing Agent
- **Purpose**: Document analysis and summarization
- **Key Learning**: Agents can effectively process and analyze documents
- **Building Blocks Used**: Role-playing, Focus, Tools

## 🚀 Next Steps:
1. **Experiment with Different LLMs**: Try GPT-4, Claude, or other local models
2. **Add More Tools**: Database connections, APIs, file operations
3. **Create Custom Tools**: Build your own specialized tools
4. **Implement Error Handling**: Add retry logic and fallback mechanisms

Keep experimenting and building! 🎉
""".format("✅" if tools_available else "⚠️")))